# 🔄 Inferência Recorrente de Vendas - CVC Lojas

## 🎯 Objetivo
Execução periódica (Semanal/Mensal) para gerar novas previsões de vendas.
Este notebook não treina modelos. Ele carrega o modelo produtivo (`All-in-One`) e gera forecast baseando-se no histórico mais recente.

## ⚙️ Fluxo de Execução
1.  **Context Loading:** Carrega os últimos 90 dias de vendas (Janela de Contexto) do Data Lake.
2.  **Model Loading:** Baixa o modelo do Unity Catalog (`Usage: Production`).
3.  **Inference:** O Wrapper `UnifiedForecaster` recebe o contexto, normaliza, prevê e desnormaliza.
4.  **Persistence:** Salva os resultados na tabela `bip_vprevisao_lojas_futuro`.


In [0]:
# --- SETUP INICIAL ---
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.getcwd())

from src.validation.config import Config
from src.validation.data import DataIngestion
from datetime import timedelta, date
import mlflow
import pyspark.sql.functions as F
import pandas as pd
from datetime import datetime

# Darts classes para Wrapper funcionar
from darts import TimeSeries

# Configs Spark
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

In [0]:
# --- 1. DEFINIÇÃO DA JANELA DE CONTEXTO ---

# Simulando "Hoje" (em prod usar date.today())
#today = date.today()
today = datetime.strptime("2025-02-01", "%Y-%m-%d").date() 

# Janela de Lookback: precisamos de histórico suficiente para os Lags do modelo (ex: 60-90 dias)
context_days = 90 
start_context = today - timedelta(days=context_days)

# Config Dinâmica
config = Config(spark)
config.DATA_START = start_context.strftime("%Y-%m-%d")
config.INGESTION_END = today.strftime("%Y-%m-%d")
config.SCHEMA = "cvc_pred"

print(f"📅 Data de Referência (Hoje): {today}")
print(f"🔎 Carregando contexto a partir de: {config.DATA_START}")

In [0]:
# --- 2. CARREGAMENTO DO CONTEXTO (SPARK) ---
# Reutilizamos a classe DataIngestion para garantir consistência nas features
ingestion = DataIngestion(spark, config)

print("   ⏳ Lendo dados históricos recentes...")
df_context_spark = ingestion.create_training_set()

# Filtro de Segurança e Seleção de Colunas
df_context_spark = df_context_spark.filter(
    F.col("DATA").between(config.DATA_START, config.INGESTION_END)
)

# Traz para Pandas (Driver) - Volume pequeno pois é só janela recente
df_context_pd = df_context_spark.toPandas()
df_context_pd['DATA'] = pd.to_datetime(df_context_pd['DATA'])

# Injeta parametro 'n' para o wrapper saber o horizonte desejado
FORECAST_HORIZON = 35
df_context_pd['n'] = FORECAST_HORIZON

print(f"   ✅ Contexto Carregado: {len(df_context_pd)} registros de vendas recentes.")
display(df_context_pd.head(5))

In [0]:
# --- 3. CARREGAMENTO DO MODELO ---

model_name = f"{config.CATALOG}.{config.SCHEMA}.cvc_lojas_forecast_production"
print(f"📥 Baixando modelo do Unity Catalog: {model_name}...")

# Em produção real, você usaria um Alias como "@Prod" ou a versão specifica
# modelo = mlflow.pyfunc.load_model(f"models:/{model_name}@Prod")
# Aqui carregamos a última versão logs
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/15")
print("   ✅ Modelo Carregado!")

In [0]:
# --- PREPARAÇÃO DE DADOS DE MERCADO ---

# 1. Carrega a tabela de suporte (a mesma usada no treino)
df_market_spark = spark.table(f"{config.CATALOG}.{config.SCHEMA}.bip_vhistorico_suporte_canal_loja")

# 2. Pivota para criar as colunas (IPCA, DOLAR...)
# Importante: O nome das colunas deve bater exatamente com o treino
df_market_wide = (df_market_spark
    .groupBy("DATA")
    .pivot("METRICAS")
    .agg(F.sum("VALOR"))
    .na.fill(0.0)
)

# 3. Converte para Pandas para fazer o merge local (já que a inferência é pandas)
pdf_market = df_market_wide.toPandas()
pdf_market['DATA'] = pd.to_datetime(pdf_market['DATA']).dt.strftime('%Y-%m-%d')

print(f"📊 Métricas de mercado carregadas: {pdf_market.columns.tolist()}")

In [0]:
# --- MONTAGEM FINAL DO CONTEXTO (MERCADO + FUTURO) ---

import numpy as np
import pandas as pd
import pyspark.sql.functions as F

# ==============================================================================
# 1. PREPARAÇÃO DE DADOS DE MERCADO (Incorporado)
# ==============================================================================
print("📊 Carregando dados de mercado (IPCA, Dólar, Feriados)...")

# Carrega a tabela de suporte
df_market_spark = spark.table(f"{config.CATALOG}.{config.SCHEMA}.bip_vhistorico_suporte_canal_loja")

# Pivota para formato Wide (Colunas: IPCA, DOLAR...)
df_market_wide = (df_market_spark
    .groupBy("DATA")
    .pivot("METRICAS")
    .agg(F.sum("VALOR"))
    .na.fill(0.0)
)

# Traz para Pandas
pdf_market = df_market_wide.toPandas()
# Garante string YYYY-MM-DD para join seguro
pdf_market['DATA'] = pd.to_datetime(pdf_market['DATA']).dt.strftime('%Y-%m-%d')


# ==============================================================================
# 2. CRIAÇÃO DO ESQUELETO FUTURO E MERGE
# ==============================================================================
print("⏳ Montando esqueleto de datas futuras para previsão...")

# Garante datetime no contexto original
df_context_pd['DATA'] = pd.to_datetime(df_context_pd['DATA'])
last_date = df_context_pd['DATA'].max()

# Gera datas futuras (+15 dias de buffer para segurança dos lags)
future_horizon_days = FORECAST_HORIZON + 15
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_horizon_days, freq='D')

# Identifica colunas estáticas para replicar (UF, Cluster, etc.)
static_cols = ['CODIGO_LOJA', 'CLUSTER_LOJA', 'SIGLA_UF', 'TIPO_LOJA', 'MODELO_LOJA']
static_cols = [c for c in static_cols if c in df_context_pd.columns]

# Pega a última "foto" de cada loja
df_stores_reference = df_context_pd.sort_values('DATA').groupby('CODIGO_LOJA')[static_cols].tail(1)

# Cross Join: Todas as Lojas x Todas as Datas Futuras
df_future_skeleton = df_stores_reference.assign(key=1).merge(
    pd.DataFrame({'DATA': future_dates, 'key': 1}), 
    on='key'
).drop('key', axis=1)

# Sinaliza que é futuro (Target NaN) e define horizonte
df_future_skeleton['TARGET_VENDAS'] = np.nan
df_future_skeleton['n'] = FORECAST_HORIZON

# Une Histórico + Futuro
df_full_timeline = pd.concat([df_context_pd, df_future_skeleton], ignore_index=True)
df_full_timeline['DATA'] = df_full_timeline['DATA'].dt.strftime('%Y-%m-%d')

# MERGE FINAL: Aplica os dados de mercado nas datas futuras
df_inference_final = pd.merge(
    df_full_timeline, 
    pdf_market, 
    on='DATA', 
    how='left'
)

# Preenche buracos eventuais no mercado com 0.0
cols_mercado = [c for c in pdf_market.columns if c != 'DATA']
df_inference_final[cols_mercado] = df_inference_final[cols_mercado].fillna(0.0)

# Ajuste de tipos
df_inference_final['CODIGO_LOJA'] = df_inference_final['CODIGO_LOJA'].astype(str)

print(f"✅ Contexto completo pronto! (Histórico + Futuro com Mercado)")
print(f"   Total de linhas: {len(df_inference_final)}")
display(df_inference_final.tail())

In [0]:
# --- PREVISÃO ---
print("🔮 Gerando previsões...")

# O modelo vai:
# 1. Receber IPCA, DOLAR, FERIADO, VENDAS...
# 2. O Wrapper vai separar: VENDAS -> Passado; RESTO -> Futuro
df_inference_final = df_inference_final.loc[:, ~df_inference_final.columns.duplicated()]

print("✅ Colunas duplicadas removidas antes da chamada do modelo.")

# 2. Agora chama a previsão
forecast_df = loaded_model.predict(df_inference_final)
display(forecast_df.head())

In [0]:
#   # --- 5. PERSISTÊNCIA (WRITE BACK) ---
#   output_table = f"{config.CATALOG}.{config.SCHEMA}.bip_vprevisao_lojas_futuro"
#   
#   print(f"💾 Salvando resultados em: {output_table}")
#   
#   try:
#       (spark.createDataFrame(forecast_df)
#        .write
#        .format("delta")
#        .mode("append") # Append histórico de previsões
#        .option("mergeSchema", "true")
#        .saveAsTable(output_table)
#       )
#       
#       spark.sql(f"OPTIMIZE {output_table}")
#       print("   ✨ Sucesso! Dados salvos e otimizados.")
#   except Exception as e:
#       print(f"❌ Erro ao salvar: {e}")